In [1]:
import os
import sys
import pandas as pd
import json
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

#### Setting up environment

In [2]:
sys.path.insert(0, os.path.abspath('..'))
from config import set_environment
set_environment()

#### Initilalize LLM

In [3]:
llm = ChatOpenAI(model_name='gpt-4-0613')

#### Load dataset

In [4]:
csv_path = './intermediate_data/04-Model/Products_Article_Matching.csv'
df = pd.read_csv(csv_path) 

#### Generate summary for one product category

In [5]:
def generate_product_summary(category):

    # Prompt template
    template = ChatPromptTemplate.from_messages([
        ('system', '''You are an expert technical writer specializing in electronic components and products.
         Create a comprehensive summary that includes 2-3 lines about what the product category is, how it is used, and why it is popular.
        Keep the summary concise, informative, and technical but accessible.'''),
        ('user', '''{category}
        Please provide a summary following the specified format.''')
    ])
    chain = template | llm | StrOutputParser()
    summary = chain.invoke({
        "category": category,
    })
    return summary

#### Generate summaries for all the product categories

In [8]:
def process_product_categories(df):

    category_column = 'Product Category'
    unique_categories = df[category_column].dropna().unique()
    product_summaries = {}
    print(f"Processing {len(unique_categories)} product categories...")
    
    for i, category in enumerate(unique_categories, 1):
        summary = generate_product_summary(category)
        product_summaries[category] = {
            "category": category,
            "summary": summary.strip()
        } 
    # Ensure output directory exists
    os.makedirs('./output', exist_ok=True)
    # Save to JSON
    with open('./intermediate_data/05-Present/Product_Descriptions.json', 'w', encoding='utf-8') as f:
        json.dump(product_summaries, f, indent=2, ensure_ascii=False)
    print(f"Product summaries saved to {'./intermediate_data/05-Present/Product_Descriptions.json'}")

    return product_summaries


#### Generate and save product category descriptions

In [9]:
if __name__ == "__main__":
    summaries = process_product_categories(df)

Processing 60 product categories...
Product summaries saved to ./intermediate_data/05-Present/Product_Descriptions.json


---

**Authors:**
[Sai Keertana Lakku](mailto:saikeertana005@gmail.com),
[Zhen Zhuang](mailto:zhuangzhen17cs@gmail.com),
[Nick Capaldini](mailto:nick.capaldini@ridethenextwave.com), Ride The Next Wave, Jun 13, 2025

---